In [1]:
import sys
import os

sys.path.insert(0, os.path.abspath("."))
sys.path.append(os.path.abspath("../../"))

In [2]:
from desc.backend import jnp, jit
from desc.grid import ConcentricGrid
from desc.equilibrium import Equilibrium
from desc.geometry import FourierRZToroidalSurface
from desc.profiles import PowerSeriesProfile
from desc.compute import data_index
from desc.compute._core import (
    compute_rotational_transform,
    compute_rotational_transform_v2,
)
from desc.transform import Transform
import numpy as np

DESC version 0.5.0+44.gbb775c3.dirty, using JAX backend, jax version=0.2.25, jaxlib version=0.1.76, dtype=float64
Using device: CPU, with 9.88 GB available memory


In [3]:
jnp.set_printoptions(precision=3, floatmode="fixed")
rng = np.random.default_rng()
L = rng.integers(low=1, high=100)
M = rng.integers(low=1, high=100)
N = rng.integers(low=1, high=100)
print(L, M, N)
grid = ConcentricGrid(L=L, N=N, M=M, node_pattern="jacobi")
# print("nodes", "             ", "spacing")
# for a, b in zip(grid.nodes, grid.spacing):
#     print(a, b)

73 41 29


## Bulk flux surface averaging test
The tests pass. timeit also shows the _surface_sums no loop algorithm is faster.

In [4]:
@jit
def _surface_sums(surf_label, unique_append_upperbound, weights):
    """
    Parameters
    ----------
    surf_label : ndarray
        The surface label. Elements of a coordinate in the collocation grid.
        i.e. grid.nodes[:, 0]
    unique_append_upperbound : ndarray
        Sorted unique elements of surf_label with the upper bound of that coordinate appended.
        i.e. grid.unique_rho + [1]
    weights : ndarray
        Node at surf_label[i]'s contribution to its surface's sum is weights[i].
        For an integral, this could be: ds * function_to_integrate.

    Returns
    -------
    ndarray
        An array of weighted sums over each surface.
        The returned array has length = len(unique_append_upperbound) - 1.
    """
    # DESIRED ALGORITHM
    # collect collocation node indices for each rho surface
    # surfaces = dict()
    # for index, rho in enumerate(surf_label):
    #     surfaces.setdefault(rho, list()).append(index)
    # integration over non-contiguous elements
    # for i, surface in enumerate(surfaces.values()):
    #     surface_sums[i] = weights[surface].sum()

    # NO LOOP IMPLEMENTATION
    # Separate collocation nodes into bins with boundaries at unique values of rho.
    # This groups nodes with identical rho values.
    # Each is assigned a weight of their contribution to the integral.
    # The elements of each bin are summed, performing the integration.
    return jnp.histogram(surf_label, bins=unique_append_upperbound, weights=weights)[0]

In [5]:
weights = np.random.random_sample(size=len(grid.nodes))

In [6]:
# %%timeit

iota_1 = np.zeros(grid.num_rho)
# DESIRED ALGORITHM
# collect collocation node indices for each rho surface
surfaces = dict()
for index, rho in enumerate(grid.nodes[:, 0]):
    surfaces.setdefault(rho, list()).append(index)
# integration over non-contiguous elements
for i, surface in enumerate(surfaces.values()):
    iota_1[i] = weights[surface].sum()

In [7]:
# %%timeit

# NO LOOP IMPLEMENTATION
bins = jnp.append(grid.nodes[grid.unique_rho_indices, 0], 1)
iota_2 = _surface_sums(grid.nodes[:, 0], bins, weights)
# bincount, bins = jnp.histogram(grid.nodes[:, 0], bins=bins)
# print(grid.nodes[grid.unique_rho_indices, 0])
# print(bincount)
# print(bins)

In [8]:
# must comment %%timeit to test assertion
print(iota_1)
print(iota_2)
assert jnp.allclose(iota_1, iota_2)

[  59.984  115.283  205.626  267.042  315.803  393.101  477.868  530.959
  596.363  648.509  743.443  797.712  862.164  926.470  997.916 1080.021
 1128.475 1186.462 1286.332 1331.837 1381.752 1403.958 1554.216 1584.757
 1667.491 1711.377 1792.302 1845.671 1921.149 1959.358 2074.140 2149.672
 2185.967 2266.464 2312.255 2392.996 2436.764]
[  59.984  115.283  205.626  267.042  315.803  393.101  477.868  530.959
  596.363  648.509  743.443  797.712  862.164  926.470  997.916 1080.021
 1128.475 1186.462 1286.332 1331.837 1381.752 1403.958 1554.216 1584.757
 1667.491 1711.377 1792.302 1845.671 1921.149 1959.358 2074.140 2149.672
 2185.967 2266.464 2312.255 2392.996 2436.764]


## Axisymmetric, vacuum, no current test
Want to test if the returned rotational transform profile is 0 when toroidal current input is 0.
This should be a good test because the bulk of the computation lies on enforcing the zero
toroidal current algorithm for the geometry of the device. And when a non-zero toroidal current is specified we just add it to the numerator because that is the delta poloidal flux term.

In [9]:
eq = Equilibrium()
# is this the canoncial way to request multiple derivatives be computed in a transform?
r_derivs = np.stack(
    (
        data_index["g_tt"]["R_derivs"][0],
        data_index["e_theta_r"]["R_derivs"][0],
        data_index["e_theta_rr"]["R_derivs"][0],
        data_index["g_tz"]["R_derivs"][0],
        data_index["g_tz"]["R_derivs"][1],
        data_index["g_tz"]["R_derivs"][2],
        data_index["e_zeta_r"]["R_derivs"][0],
        data_index["e_zeta_r"]["R_derivs"][1],
        data_index["e_zeta_rr"]["R_derivs"][0],
        data_index["e_zeta_rr"]["R_derivs"][1],
        data_index["R_rrz"]["R_derivs"][0],
    )
)
l_derivs = np.stack(
    (
        data_index["lambda_t"]["L_derivs"][0],
        data_index["lambda_rt"]["L_derivs"][0],
        data_index["lambda_rrt"]["L_derivs"][0],
        data_index["lambda_z"]["L_derivs"][0],
        data_index["lambda_rz"]["L_derivs"][0],
        data_index["lambda_rrz"]["L_derivs"][0],
    )
)
R_transform = Transform(grid, eq.R_basis, derivs=r_derivs, build=True)
Z_transform = Transform(grid, eq.Z_basis, derivs=r_derivs, build=True)
L_transform = Transform(grid, eq.L_basis, derivs=l_derivs, build=True)

In [10]:
I_l = np.array([0, 0])
toroidal_current = PowerSeriesProfile(params=[1, 1.5], modes=[0, 2], grid=grid)

data = compute_rotational_transform_v2(
    eq.R_lmn,
    eq.Z_lmn,
    eq.L_lmn,
    R_transform,
    Z_transform,
    L_transform,
    eq.Psi,
    I_l=I_l,
    toroidal_current=toroidal_current,
)

In [11]:
assert data["iota"].size == grid.nodes[:, 0].size
assert jnp.allclose(data["iota"], 0)